# Privacy-Preserving CNN

## Preparations

### modules

In [ ]:
import numpy as np  # handling vectors and matrices
import matplotlib.pyplot as plt  # plotting
from keras.models import load_model  # reload pretrained tf models
from pnn_functions import (train_tf_model, pred_tf_model,
                           train_pt_model, pred_pt_model)  # plain neural net functions
from evaluation_functions import evaluate_forecasts  # evaluation functions
from preproc_functions import list_combine  # own functions
from nn_arguments import Arguments  # nn options
from neural_nets import TFUNET, TFMNET, PTUNET, PTMNET  # own defined neural nets as classes

from torchvision import models

# pysyft
import syft as sy
import torch  # pytorch
hook = sy.TorchHook(torch)  # hooking pysyft into pytorch
import torch.nn.functional as F
import torch.nn as nn

### options

In [ ]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### datasets

#### test set

In [ ]:
# We need only the test-sets - Model is already trained
#univariate
test_Xu = np.load(dir_data+'test_Xs.npy')[:,:,:1]

In [ ]:
#mutlivariate
test_Xm = np.load(dir_data+'test_Xs.npy')

In [ ]:
#Target Testset
test_y = np.load(dir_data+'test_ys.npy')

In [ ]:
#Reshape Dimensions for Pytorch for Dataset
test_Xu = torch.from_numpy(np.array(test_Xu.reshape(test_Xu.shape[0],
                                                        test_Xu.shape[2],
                                                        test_Xu.shape[1]),
                                        dtype='float32'))

In [ ]:
#Check Dimension Data
test_Xu.shape

In [ ]:
#Check Dimension Data
test_Xm.shape

In [ ]:
#Check Dimension Target
test_y.shape

In [ ]:
#Load Data over DataLoader
test_loaderXu = torch.utils.data.DataLoader(
    np.load(dir_data+'test_Xs.npy')[:,:,:1])
print (test_loaderXu)

test_loadery = torch.utils.data.DataLoader(
    np.load(dir_data+'test_ys.npy'))
print (test_loadery)

## Models

### Options

In [ ]:
args = Arguments()
# overwrite timesteps depending on data
args.n_timesteps = test_Xu.shape[1]

In [ ]:
test_Xu.shape[1]

### load pretrained weights

#### tensorflow

#### pytorch

In [ ]:
# create net objects
model_ptu = PTUNET(args.n_features_u, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)
model_ptm = PTMNET(args.n_features_m, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)

In [ ]:
# load weights
model_ptu.load_state_dict(torch.load(dir_data+'models/weights_ptu.pt'), strict=False)
model_ptm.load_state_dict(torch.load(dir_data+'models/weights_ptm.pt'), strict=False)

## Distribute data and model between parties (all encrypted)

In [ ]:
# different parties
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

SMPC works with integers so we need to convert floats to int via fix_precision.

In [ ]:
# share test data
testData = torch.tensor(test_loaderXu).fix_precision().share(alice, bob, charlie, crypto_provider=crypto_provider)

In [ ]:
testTarget = torch.tensor(test_loadery).fix_precision().share(alice, bob, charlie, crypto_provider=crypto_provider)

In [ ]:
list(test_loaderXu)[0].share()

In [ ]:
#show dataset
bob._objects

In [ ]:
#show dataset
charlie._objects

In [ ]:
alice._objects

In [ ]:
# share model
model_ptu.fix_precision().share(alice, bob, charlie, crypto_provider=crypto_provider)

In [ ]:
def test(args, model, data, target):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
       # for data, target in a, b:
        output = model(data)
        pred = output.argmax(dim=1) 
        n_correct_priv += pred.eq(target.view_as(pred)).sum()
        n_total += args.test_batch_size
        print (pred)
        print(output)
#The following test function performs the encrypted evaluation. The model weights, the data inputs, the prediction and the target used for scoring are all encrypted!

# However as you can observe, the syntax is very similar to normal PyTorch testing! Nice!

# The only thing we decrypt from the server side is the final score at the end of our 200 items batches to verify predictions were on average good.      
        n_correct = n_correct_priv.copy().get().float_precision().long().item()

        print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
            n_correct, n_total,
            100. * n_correct / n_total))

In [ ]:
#Idea 1: Without objectize the share to Virtual Worker
test(args, model_ptu, test_loaderXu , test_loadery)

In [ ]:
#Idea 1: with obejctize the share to Virtual Worker
test(args, model_ptu, testData , testTarget)

End! --- Please ignore the following Code --- just not to loose this Options

In [ ]:
#not nessecary now
def test(modelptu, X, y):
    modelptu.eval()
    with torch.no_grad():
        pred_1 = modelptu(X)
        score, scores = evaluate_forecasts(y, pred)
    return score

In [ ]:
#Dont do it! - Just for Testing
bobs_model = modelu.copy().send(bob)
alices_model = modelu.copy().send(alice)
charlie_model = modelu.copy().send(charlie)